# Vektordatenbank Suche
Ein Notebook nur spezialisiert auf die Suche in der Vektordatenbank

In [20]:
from classes import *

workers = get_test_data()

Einige spezialisierte Test-Mitarbeiter hinzufügen

In [21]:
test_mitarbeiter = [
    MitarbeiterSkills(
        name="Boris Backend",
        technologien=[
            TechnologieWissen(name="Java", letzte_verwendung=0),
            TechnologieWissen(name="Docker", letzte_verwendung=0),
            TechnologieWissen(name="SQL", letzte_verwendung=0),
            TechnologieWissen(name="MongoDB", letzte_verwendung=0),
            TechnologieWissen(name="PostgreSQL", letzte_verwendung=0),
            TechnologieWissen(name="MySQL", letzte_verwendung=0)
        ]
    ),
    MitarbeiterSkills(
        name="Fritz Frontend",
        technologien=[
            TechnologieWissen(name="JavaScript", letzte_verwendung=0),
            TechnologieWissen(name="TypeScript", letzte_verwendung=0),
            TechnologieWissen(name="React", letzte_verwendung=0),
            TechnologieWissen(name="Vue.js", letzte_verwendung=0),
            TechnologieWissen(name="Angular", letzte_verwendung=0)
        ]
    ),
    MitarbeiterSkills(
        name="Tim Technik",
        technologien=[
            TechnologieWissen(name="Azure", letzte_verwendung=0),
            TechnologieWissen(name="AWS", letzte_verwendung=0),
            TechnologieWissen(name="GCP", letzte_verwendung=0),
            TechnologieWissen(name="Kubernetes", letzte_verwendung=0),
            TechnologieWissen(name="Docker", letzte_verwendung=0)
        ]
    )
]

# Füge die Test-Mitarbeiter zu den bestehenden Mitarbeitern hinzu
workers.extend(test_mitarbeiter)

In [22]:
from openai import OpenAI

client = OpenAI()

def embedd(data):
    return client.embeddings.create(input=data, model="text-embedding-3-small").data[0].embedding

In [23]:
import os
import numpy as np

EMBEDDINGS_PATH = "embeddings.npy"

embedded_data = []

if os.path.exists(EMBEDDINGS_PATH):
    print("Lade gespeicherte Embeddings...")
    embedded_data = np.load(EMBEDDINGS_PATH)
else:
    print("Berechne Embeddings und speichere sie...")
    embedded_data = []
    for worker in workers:
        embedded_data.append(embedd(worker.to_embedding()))
    embedded_data = np.array(embedded_data)
    np.save(EMBEDDINGS_PATH, embedded_data)

Berechne Embeddings und speichere sie...


In [24]:
import faiss
import numpy as np

dim = len(embedded_data[0])
index = faiss.IndexFlatL2(dim)

embeddings_np = np.array(embedded_data).astype("float32")
index.add(embeddings_np)

## Notitz
Es gibt 2 Ansätze, die man noch probieren könnte.

1. Alle Daten vor dem Embedding in Fließtext konvertieren
2. Query-Vektor in Stichpunkte bzw. konkrete Technologie suche Konvertieren (siehe Excalidraw) 

In [ ]:
from embeddings_utils import get_query_embedding

query = "Erfahrung mit Backend Technologien"
query_vector = get_query_embedding(query, embedd)

search_vector = np.array([query_vector]).astype('float32')

erg_anzahl = 5

D, indices = index.search(search_vector, erg_anzahl)

print(f"Distances: {D}")
print(f"Indices: {indices}")

for i in indices[0]:
    print(workers[i])

Distances: [[1.1303233 1.1606724 1.2330351 1.2354889 1.2402079]]
Indices: [[225 162  96 190 206]]
Mitarbeiter:
  Name: Alina Uhlig
  Skills:
    Java: vor 262 Tagen
    JIRA: vor 110 Tagen
Mitarbeiter:
  Name: Tarek Scholz
  Skills:
    Unity: vor 706 Tagen
    Jenkins: vor 368 Tagen
Mitarbeiter:
  Name: Henrik Vogt
  Skills:
    Unity: vor 445 Tagen
    Java: vor 259 Tagen
Mitarbeiter:
  Name: Kevin Heinemann
  Skills:
    JIRA: vor 233 Tagen
    Confluence: vor 469 Tagen
    TypeScript: vor 52 Tagen
    Ruby: vor 569 Tagen
    Terraform: vor 220 Tagen
Mitarbeiter:
  Name: Jette Mathis
  Skills:
    JIRA: vor 85 Tagen
    Git: vor 320 Tagen
    Swift: vor 284 Tagen
    Confluence: vor 658 Tagen
